In [1]:
import pandas as pd
import numpy as np
from keras.datasets import boston_housing
from keras import layers, models, optimizers

In [2]:
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

In [3]:
train_data.shape, '-', train_labels.shape, '-', train_labels[0], '-', train_data[0]

((404, 13),
 '-',
 (404,),
 '-',
 15.2,
 '-',
 array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
         91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
         18.72   ]))

# Normalizar
Se normaliza ya que los valores del train_data son muy diferentes

In [4]:
mean = train_data.mean(axis=0)
train_data = train_data - mean
std = train_data.std(axis=0)
train_data = train_data/std

test_data = test_data - mean
test_data = test_data/std
mean, mean.shape, std

(array([3.74511057e+00, 1.14801980e+01, 1.11044307e+01, 6.18811881e-02,
        5.57355941e-01, 6.26708168e+00, 6.90106436e+01, 3.74027079e+00,
        9.44059406e+00, 4.05898515e+02, 1.84759901e+01, 3.54783168e+02,
        1.27408168e+01]),
 (13,),
 array([9.22929073e+00, 2.37382770e+01, 6.80287253e+00, 2.40939633e-01,
        1.17147847e-01, 7.08908627e-01, 2.79060634e+01, 2.02770050e+00,
        8.68758849e+00, 1.66168506e+02, 2.19765689e+00, 9.39946015e+01,
        7.24556085e+00]))

In [5]:
train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [15]:
def build_model_regression(lr_var, input_data):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(input_data,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1)) # Capa lineal, no necesita funcion de activacion
    # Compilar modelo
    model.compile(optimizer=optimizers.RMSprop(lr=lr_var), loss='mse', metrics=['mae'])
    return model

In [16]:
# Iteracion sobre set de validacion
k=4
num_val_samples = len(train_data)//k
num_epoch = 86
all_history = []

In [17]:
valor = 2
valor * num_val_samples

202

In [18]:
for i in range(k):
    print('Fold: ', i)
    val_data = train_data[i*num_val_samples : (i+1)*num_val_samples]
    val_targets = train_labels[i*num_val_samples : (i+1)*num_val_samples]

    partial_train_data = np.concatenate([
        train_data[:i*num_val_samples],
        train_data[(i+1)*num_val_samples:]
    ], axis=0)

    partial_train_targets = np.concatenate([
        train_labels[:i*num_val_samples],
        train_labels[(i+1)*num_val_samples:]
    ], axis=0)

    model = build_model_regression(0.001, 13)
    history = model.fit(partial_train_data, partial_train_targets, 
                        epochs = num_epoch,
                        batch_size=16, # take # of sample
                        validation_data = (val_data, val_targets),
                        verbose=0
                        )
    all_history.append(history.history['val_mae'])

Fold:  0


c:\Users\Rreyes\Anaconda3\envs\py39\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Fold:  1
Fold:  2
Fold:  3


In [10]:
len(all_history[0]) #, all_history.shape

500

In [14]:
pd.DataFrame(all_history)
# n iteraciones * n epoch

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,19.230764,15.956750,11.777609,7.716180,4.984888,4.068442,3.586604,3.418650,3.227356,3.081834,...,2.275151,2.180610,2.291591,2.255949,2.318610,2.263773,2.271024,2.309588,2.172964,2.253187
1,18.833363,14.994183,10.358963,7.025309,5.242038,4.344861,3.736925,3.419858,3.236396,3.158445,...,3.114483,2.946495,3.069580,3.047099,2.941113,3.060998,3.159113,3.267203,3.151031,3.019681
2,18.621962,15.255015,10.923498,7.639626,5.837283,4.817229,4.193263,3.723631,3.401771,3.303784,...,2.720668,2.681542,2.646420,2.697246,2.709988,2.715971,2.699645,2.639695,2.642933,2.745013
3,21.085688,17.449398,12.760543,9.161733,7.424747,6.135283,5.249290,4.682052,4.228546,3.990954,...,2.176543,2.186292,2.104284,2.588962,2.374708,2.169191,2.071869,2.387677,2.281885,2.413305


In [11]:
all_mae_avg = pd.DataFrame(all_history).mean(axis=0)
all_mae_avg

0      19.442944
1      15.913836
2      11.455153
3       7.885712
4       5.872239
         ...    
495     2.552483
496     2.550413
497     2.651041
498     2.562203
499     2.607797
Length: 500, dtype: float64

In [12]:
import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(10, 10))
# plt.plot(range(1, len(all_mae_avg[15:])+1), all_mae_avg[15:])
# plt.show()

In [13]:
model.evaluate(test_data, test_labels)

4/4 [==============================] - 0s 2ms/step - loss: 16.7508 - mae: 2.5997


[16.75077247619629, 2.599686622619629]